In [ ]:
import pandas as pd
from transformers import pipeline

#If we want to add the progress bar we will need to add this
from tqdm.auto import tqdm
tqdm.pandas()

In [27]:
starwars_df = pd.read_csv('./sample_train.csv',index_col='idx')
starwars_df.drop(columns=['page','block'],inplace=True)
starwars_df['text'] = starwars_df.apply(lambda x: x.term + ' ' + x.definition,axis=1)
starwars_df.head()

,term,definition,text
idx,,,
455,All Stars Burn as One,The official anthem of the Galactic Republic.,All Stars Burn as One The official anthem of t...
1534,bocatt,"A tusked, leather-skinned predator found on Ta...","bocatt A tusked, leather-skinned predator foun..."
3142,Daughters of Allya,The name adopted by the Dathomir Witches. Ther...,Daughters of Allya The name adopted by the Dat...
322,Aidus,A Rattataki guard who served Asajj Ventress du...,Aidus A Rattataki guard who served Asajj Ventr...
8710,Insurrection,This branch of the Pentastar Alignment’s Chamb...,Insurrection This branch of the Pentastar Alig...


In [28]:
classifier = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')
candidate_labels = ['Ship','Creature','Character','Planet','Device','Specie','Weapon','Location','Organization']

Device set to use cuda:0


In [29]:
def classify_with_model(text):
    results = classifier(text,candidate_labels,multi_label=True)
    
    confident_labels = [
        label for label, score in zip(results['labels'],results['scores']) if score > 0.5
    ]
    
    return confident_labels if confident_labels else ['Unknown']

In [30]:

#To show the progress we will need to use this line
starwars_df['classification'] = starwars_df['text'].progress_apply(classify_with_model)

#If we do not to see the progress we can use this line instead
#starwars_df['classification'] = starwars_df['text'].apply(classify_with_model)


  0%|          | 0/1000 [00:00<?, ?it/s]

In [32]:
starwars_df.head(15)

,term,definition,text,classification
idx,,,,
455,All Stars Burn as One,The official anthem of the Galactic Republic.,All Stars Burn as One The official anthem of t...,[Unknown]
1534,bocatt,"A tusked, leather-skinned predator found on Ta...","bocatt A tusked, leather-skinned predator foun...",[Creature]
3142,Daughters of Allya,The name adopted by the Dathomir Witches. Ther...,Daughters of Allya The name adopted by the Dat...,[Planet]
322,Aidus,A Rattataki guard who served Asajj Ventress du...,Aidus A Rattataki guard who served Asajj Ventr...,[Character]
8710,Insurrection,This branch of the Pentastar Alignment’s Chamb...,Insurrection This branch of the Pentastar Alig...,[Unknown]
8802,Invisible Island,"A legendary island of untold riches, a virtual...",Invisible Island A legendary island of untold ...,[Unknown]
3042,Dark Force (1),See Katana fleet.,Dark Force (1) See Katana fleet.,[Unknown]
7630,Hockaleg,"A planet in the Patriim system, it served as a...","Hockaleg A planet in the Patriim system, it se...","[Planet, Location]"
15096,Qiemal,A primitive sect of the Qieg. While most of th...,Qiemal A primitive sect of the Qieg. While mos...,"[Location, Character, Ship]"
